In [3]:
import gym
import collections
import torch
import torch.optim as optim

import collections

In [2]:
ENV_NAME = "FrozenLake-v0"
GAMMA = 0.9
TEST_EPISODES = 20

## Value Learning

In [13]:
class Agent:
    def __init__(self):
        self.env = gym.make(ENV_NAME)
        self.state = self.env.reset()
        self.rewards = collections.defaultdict(float)
        self.transits = collections.defaultdict(collections.Counter)
        self.values = collections.defaultdict(float)
    
    def play_n_random_steps(self,count):
        for _ in range(count):
            action = self.env.action_space.sample()
            new_state, reward, is_done, _ = self.env.step(action)
            self.rewards[(self.state, action, new_state)] = reward
            self.transits[(self.state, action)][new_state] += 1
            self.state = self.env.reset() if is_done else new_state
    
    def calc_action_value(self, state, action):
        target_counts = self.transits[(state,action)]
        total = sum(target_counts.values())
        action_value = 0.0
        for tgt_state, count in target_counts.items():
            reward = self.rewards[(state, action, tgt_state)]
            action_value += (count / total) * (reward + GAMMA * self.values[tgt_state])
        return action_value
    
    def select_action(self, state):
        best_action, best_value = None, None
        for action in range(self.env.action_space.n):
            action_value = self.calc_action_value(state,action)
            if best_value is None or best_value < action_value:
                best_value = action_value
                best_action = action
        return best_action
    
    def play_episode(self, env):
        total_reward = 0.0
        state = env.reset()
        while True:
            action = self.select_action(state)
            new_state, reward, is_done, _ = env.step(action)
            self.rewards[(state,action,new_state)] = reward
            self.transits[(state, action)][new_state] += 1
            total_reward += reward
            if is_done:
                break
            state = new_state
        return total_reward
    
    def value_iteration(self):
        for state in range(self.env.observation_space.n):
            state_values = [self.calc_action_value(state, action)
                            for action in range(self.env.action_space.n)]
            self.values[state] = max(state_values)

In [14]:
test_env = gym.make(ENV_NAME)
agent = Agent()

In [15]:
iter_no = 0
best_reward = 0.0
while True:
    iter_no += 1
    agent.play_n_random_steps(100)
    agent.value_iteration()
    
    reward = 0.0
    for _ in range(TEST_EPISODES):
        reward += agent.play_episode(test_env)
    reward /= TEST_EPISODES
    if reward > best_reward:
        print('best reward updated %.3f -> %.3f, at iter %d'%(best_reward, reward, iter_no))
        best_reward = reward
    if reward > 0.8:
        print('Solved in %d iterations'% iter_no)
        break

best reward updated 0.000 -> 0.100, at iter 6
best reward updated 0.100 -> 0.150, at iter 7
best reward updated 0.150 -> 0.250, at iter 9
best reward updated 0.250 -> 0.350, at iter 12
best reward updated 0.350 -> 0.500, at iter 15
best reward updated 0.500 -> 0.850, at iter 16
Solved in 16 iterations


## Q-Learning

In [16]:
class Agent:
    def __init__(self):
        self.env = gym.make(ENV_NAME)
        self.state = self.env.reset()
        self.rewards = collections.defaultdict(float)
        self.transits = collections.defaultdict(collections.Counter)
        self.values = collections.defaultdict(float)
    
    # play randomly
    def play_n_random_steps(self,count):
        for _ in range(count):
            action = self.env.action_space.sample()
            new_state, reward, is_done, _ = self.env.step(action)
            self.rewards[(self.state, action, new_state)] = reward
            self.transits[(self.state, action)][new_state] += 1
            self.state = self.env.reset() if is_done else new_state
    
    # finds the action that maxs the Q value
    def select_action(self, state):
        best_action, best_value = None, None
        for action in range(self.env.action_space.n):
            action_value = self.values[(state,action)]
            if best_value is None or best_value < action_value:
                best_value = action_value
                best_action = action
        return best_action
    
    # play under the policy
    def play_episode(self, env):
        total_reward = 0.0
        state = env.reset()
        while True:
            action = self.select_action(state)
            new_state, reward, is_done, _ = env.step(action)
            self.rewards[(state,action,new_state)] = reward
            self.transits[(state, action)][new_state] += 1
            total_reward += reward
            if is_done:
                break
            state = new_state
        return total_reward
    
    def Q_iteration(self):
        for state in range(self.env.observation_space.n):
            for action in range(self.env.action_space.n):
                action_value = 0.0
                target_counts = self.transits[(state,action)]
                total = sum(target_counts.values())
                for tgt_state, count in target_counts.items():
                    reward = self.rewards[(state,action,tgt_state)]
                    best_action = self.select_action(tgt_state)
                    action_value += (count/total)*(reward + GAMMA*self.values[(tgt_state,best_action)])
                self.values[(state,action)] = action_value

In [18]:
test_env = gym.make(ENV_NAME)
agent2 = Agent()

iter_no = 0
best_reward = 0.0
while True:
    iter_no += 1
    agent2.play_n_random_steps(100)
    agent2.Q_iteration()
    
    reward = 0.0
    for _ in range(TEST_EPISODES):
        reward += agent2.play_episode(test_env)
    reward /= TEST_EPISODES
    if reward > best_reward:
        print('best reward updated %.3f -> %.3f, at iter %d'%(best_reward, reward, iter_no))
        best_reward = reward
    if reward > 0.8:
        print('Solved in %d iterations'% iter_no)
        break

best reward updated 0.000 -> 0.350, at iter 6
best reward updated 0.350 -> 0.500, at iter 14
best reward updated 0.500 -> 0.550, at iter 15
best reward updated 0.550 -> 0.600, at iter 22
best reward updated 0.600 -> 0.700, at iter 23
best reward updated 0.700 -> 0.800, at iter 24
best reward updated 0.800 -> 0.850, at iter 84
Solved in 84 iterations


In [20]:
agent2.values

defaultdict(float,
            {(0, 0): 0.06706148429665608,
             (0, 1): 0.0651552097081867,
             (0, 2): 0.06474703340435528,
             (0, 3): 0.058267554792775414,
             (4, 0): 0.08973405121307176,
             (4, 1): 0.06949883334056026,
             (4, 2): 0.0627169593514071,
             (4, 3): 0.047546521879407,
             (1, 0): 0.03688509178499911,
             (1, 1): 0.04307962017304282,
             (1, 2): 0.0418807523715567,
             (1, 3): 0.06002498509763962,
             (5, 0): 0.0,
             (5, 1): 0.0,
             (5, 2): 0.0,
             (5, 3): 0.0,
             (2, 0): 0.07294656716979389,
             (2, 1): 0.06631833494157095,
             (2, 2): 0.07260971743316598,
             (2, 3): 0.056462896968233554,
             (3, 0): 0.03853009158374472,
             (3, 1): 0.036027242065178974,
             (3, 2): 0.031774974042421154,
             (3, 3): 0.05491656751288794,
             (7, 0): 0.0,
            

## Q-learning with exponential smoothing

In [28]:
ENV_NAME = "FrozenLake-v0"
GAMMA = 0.9
ALPHA = 0.2
TEST_EPISODES = 20

In [35]:
class Agent:
    def __init__(self):
        self.env = gym.make(ENV_NAME)
        self.state = self.env.reset()
        self.values = collections.defaultdict(float)
    
    # random step
    def sample_env(self):
        action = self.env.action_space.sample()
        old_state = self.state
        new_state, reward, is_done, _ = self.env.step(action)
        self.state = self.env.reset() if is_done else new_state
        return(old_state, action, reward, new_state)
    
    # find max_a Q(a,s->s')
    def best_value_and_action(self,state):
        best_value, best_action = None, None
        for action in range(self.env.action_space.n):
            action_value = self.values[(state,action)]
            if best_action is None or best_value < action_value:
                best_value = action_value
                best_action = action
        return(best_value, best_action)
    
    def value_update(self, s, a, r, next_s):
        max_Q, _ = self.best_value_and_action(next_s)
        new_Q = r + GAMMA*max_Q
        old_val = self.values[(s,a)]
        self.values[(s,a)] = old_val*(1-ALPHA)+new_Q*ALPHA
    
    def play_episode(self, env):
        total_reward = 0.0
        state = env.reset()
        while True:
            _, action = self.best_value_and_action(state) # choose the a that maxes the Q(s,a)
            new_state, reward, is_done, _ = env.step(action)
            total_reward += reward
            if is_done:
                break
            state = new_state
        return(total_reward)

In [36]:
test_env = gym.make(ENV_NAME)
agent = Agent()

iter_no = 0
best_reward = 0.0
while True:
    iter_no += 1
    # random step
    s,a,r, next_s = agent.sample_env()
    agent.value_update(s,a,r,next_s)
    
    reward = 0.0
    for _ in range(TEST_EPISODES):
        reward += agent.play_episode(test_env)
    reward /= TEST_EPISODES
    if reward > best_reward:
        print('Best reward updated %.3f -> %.3f'%(best_reward,reward))
        best_reward = reward
    if reward > 0.8:
        print('Solved in %d iterations!'%iter_no)
        break

Best reward updated 0.000 -> 0.200
Best reward updated 0.200 -> 0.250
Best reward updated 0.250 -> 0.350
Best reward updated 0.350 -> 0.400
Best reward updated 0.400 -> 0.500
Best reward updated 0.500 -> 0.550
Best reward updated 0.550 -> 0.600
Best reward updated 0.600 -> 0.650
Best reward updated 0.650 -> 0.700
Best reward updated 0.700 -> 0.800
Best reward updated 0.800 -> 0.900
Solved in 3149 iterations!


In [37]:
agent.values

defaultdict(float,
            {(1, 0): 0.0031576977853526806,
             (1, 1): 0.007575567640188774,
             (1, 2): 0.006107547210362401,
             (1, 3): 0.008785176810668892,
             (0, 3): 0.00884489255862422,
             (0, 0): 0.01029908224689036,
             (0, 1): 0.009135396258147903,
             (0, 2): 0.009344848842067256,
             (4, 0): 0.01129816175324733,
             (4, 1): 0.009170133087938738,
             (4, 2): 0.010706019969529332,
             (4, 3): 0.007473541294029199,
             (8, 0): 0.0074701365096461435,
             (8, 1): 0.01686509086893119,
             (8, 2): 0.011005526833541973,
             (8, 3): 0.018090601670047553,
             (5, 0): 0.0,
             (5, 1): 0.0,
             (5, 2): 0.0,
             (5, 3): 0.0,
             (2, 0): 0.008632007601120182,
             (2, 1): 0.011718582351450114,
             (2, 2): 0.009723281555156622,
             (2, 3): 0.003952769130538033,
             (6, 0)